In [1]:
%%datalabframework getfilename

In [2]:
import datalabframework as dlf
logger = dlf.log.initLogger(__name__, kafka_topic="datalab", kafka_servers="kafka:9092")

2017-04-30 14:25:29,080 - jovyan - __main__ - INFO - init - {'notebook': {'filepath': '/home/jovyan/work/notebooks/flows', 'filename': 'lr.ipynb'}, 'datalab': {'framework': '0.1'}, 'project': {'main': 'main.ipynb', 'rootpath': '/home/jovyan/work/notebooks'}}


In [3]:
DATA_ROOT = '/home/jovyan/work/data'
d = {   
    'input_sample' : 1.0,
    'input_source' : DATA_ROOT + '/titanic/data/raw/train.csv'
}
p = dlf.params.config_fromdict(d)

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName(dlf.project.filename()).getOrCreate()
spark.version

'2.1.0'

In [6]:
from extract import extract
from features import features
from models import lr
from validate import validate

importing Jupyter notebook from /home/jovyan/work/notebooks/extract/extract.ipynb
importing Jupyter notebook from /home/jovyan/work/notebooks/features/features.ipynb
importing Jupyter notebook from /home/jovyan/work/notebooks/models/lr.ipynb


In [7]:
df = extract.load_csv(spark, p.input_source)
df = extract.clean_dataset(df)
df = features.inpute_simple(df, 'Embarked')
df = features.impute_lr(df, 'PassengerId', 'Age', ['Sex', 'Pclass', 'SibSp', 'Parch', 'Embarked', 'Fare']) 
df = features.feature_title(df)
df = features.feature_vector(df, 'PassengerId', 'Survived', 
                  ['Pclass','Sex','Age', 'SibSp', 'Parch', 'Fare','Embarked','Title'])

In [8]:
df.show(5, truncate=False)

+-----------+----------------------------------------------------+-----+
|PassengerId|features                                            |label|
+-----------+----------------------------------------------------+-----+
|1          |(24,[0,1,2,3,5,6,8],[3.0,1.0,22.0,1.0,7.25,1.0,1.0])|0.0  |
|2          |(24,[0,2,3,5,7,10],[1.0,38.0,1.0,71.2833,1.0,1.0])  |1.0  |
|3          |(24,[0,2,5,6,9],[3.0,26.0,7.925,1.0,1.0])           |1.0  |
|4          |(24,[0,2,3,5,6,10],[1.0,35.0,1.0,53.1,1.0,1.0])     |1.0  |
|5          |(24,[0,1,2,5,6,8],[3.0,1.0,35.0,8.05,1.0,1.0])      |0.0  |
+-----------+----------------------------------------------------+-----+
only showing top 5 rows



In [9]:
model = lr.lr_model(df)
metric_name, metric = validate.auc(df,model)

In [10]:
print("Evaluation {} : {}".format(metric_name, metric))

Evaluation areaUnderROC : 0.8742077567933189
